In [1]:
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
import time
from gseapy.plot import barplot, dotplot

wk_dir = '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-4/data/4-20_enrichr_plot/'

In [2]:
# add file path
# add df.name
# append to list
# DEG_kd_B14_vs_TGFb = '/Users/jplab/Desktop/RNA_seq_FOXA1KO_scramble_20211213/downstream/data/raw_count_flow/deg/DEG_kd_B14_vs_TGFb.tsv'
# DEG_nc_B14_vs_TGFb = '/Users/jplab/Desktop/RNA_seq_FOXA1KO_scramble_20211213/downstream/data/raw_count_flow/deg/DEG_nc_B14_vs_TGFb.tsv'
# DEG_2d_data = '/Users/jplab/Desktop/2021-12/data/12-29_2d_rna_deg/2umol_B14_vs_TGFb_DEG.tsv'

DEG_B14_KD_vs_NC = '/Users/jplab/Desktop/snakepipes_Data/results_mRNA_seq_FOXA1_KD_NC_2022221/DESeq2_samplesheet_FOXA1-B14-NCvsKD/DEseq_basic_DEresults_counts_DESeq2.normalized_gt10_noVersion.tsv'
# DEG_TGF_KD_vs_NC = '/Users/jplab/Desktop/snakepipes_Data/results_mRNA_seq_FOXA1_KD_NC_2022221/DESeq2_samplesheet_FOXA1-TGFb-NCvsKD/DEseq_basic_DEresults_counts_DESeq2.normalized_noVersion.tsv'


df_B14_KD_vs_NC= pd.read_csv(DEG_B14_KD_vs_NC,header=0, sep="\t",index_col=False).dropna()
df_B14_KD_vs_NC.name = 'df_B14_KD_vs_NC'

# df_TGF_KD_vs_NC= pd.read_csv(DEG_B14_KD_vs_NC,header=0, sep="\t",index_col=False).dropna()
# df_TGF_KD_vs_NC.name = 'df_TGF_KD_vs_NC'

# df_TGF_KD_vs_NC= pd.read_csv(DEG_B14_KD_vs_NC,header=0, sep="\t",index_col=False).dropna()
# df_TGF_KD_vs_NC.name = 'df_TGF_KD_vs_NC'
# df_TGF_KD_vs_NC = pd.read_csv(DEG_TGF_KD_vs_NC,header=0, sep="\t",index_col=False).dropna()
# df_TGF_KD_vs_NC.name = 'df_TGF_KD_vs_NC'
# df_2d_data = pd.read_csv(DEG_2d_data,header=0, sep="\t",index_col=False)
# df_2d_data.name = 'df_2d_data'

df_list = []
# df_list.append(df_B14_KD_vs_NC)
df_list.append(df_B14_KD_vs_NC)
# df_list.append(df_2d_data)


In [6]:
# first use dict to save loop filter results
dict = {}
for df in df_list:
    print(df.name)
    df_name = df.name

    # dict[df_name + '_UP.LOGFC0.3_genes'] = df.query("log2FoldChange >= 0.3")
    dict[df_name + '_UP_genes'] = df.query("Status == 'UP'")

    # dict[df_name + '_DOWN_genes'] = df.query("Status == 'DOWN'")
    # dict[df_name + '_UP_and_DOWN_genes'] = df.query("Status == 'UP' or Status == 'DOWN'")
    # dict[df_name + '_UP_DOWN_NONE_genes'] = df.query("Status == 'UP' or Status == 'DOWN' or Status == 'None'")
print(dict.keys())

# then loop dict
dict_gene_list = {}
for df_name, df in dict.items():
    # sort by fold change
    df = df.sort_values(by=['log2FoldChange'], ascending=False)

    # drop dup in symbol column
    df_ori_row_num = len(df.index)
    df = df.drop_duplicates(subset=['external_gene_name'], keep='first')
    df_dupDrop_row_num = len(df.index)
    print(df_name + ': ' + str(df_ori_row_num - df_dupDrop_row_num) + ' rows of dup are dropped')

    # must drop na, or will throw error in enrichr method
    df = df['external_gene_name'].dropna()
    df_dropNa_row_num = len(df.index)
    print(df_name + ': ' + str(df_dupDrop_row_num - df_dropNa_row_num) + ' rows of NA are dropped')

    # convert to list
    dict_gene_list[df_name.split('df_')[1]] = df.tolist()



df_B14_KD_vs_NC
dict_keys(['df_B14_KD_vs_NC_UP_genes'])
df_B14_KD_vs_NC_UP_genes: 0 rows of dup are dropped
df_B14_KD_vs_NC_UP_genes: 0 rows of NA are dropped


In [4]:
# get enrichr lib name from GSEApy
lib_names = gp.get_library_name()
lib_names
print(lib_names)

['ARCHS4_Cell-lines', 'ARCHS4_IDG_Coexp', 'ARCHS4_Kinases_Coexp', 'ARCHS4_TFs_Coexp', 'ARCHS4_Tissues', 'Achilles_fitness_decrease', 'Achilles_fitness_increase', 'Aging_Perturbations_from_GEO_down', 'Aging_Perturbations_from_GEO_up', 'Allen_Brain_Atlas_10x_scRNA_2021', 'Allen_Brain_Atlas_down', 'Allen_Brain_Atlas_up', 'Azimuth_Cell_Types_2021', 'BioCarta_2013', 'BioCarta_2015', 'BioCarta_2016', 'BioPlanet_2019', 'BioPlex_2017', 'CCLE_Proteomics_2020', 'CORUM', 'COVID-19_Related_Gene_Sets', 'COVID-19_Related_Gene_Sets_2021', 'Cancer_Cell_Line_Encyclopedia', 'CellMarker_Augmented_2021', 'ChEA_2013', 'ChEA_2015', 'ChEA_2016', 'Chromosome_Location', 'Chromosome_Location_hg19', 'ClinVar_2019', 'DSigDB', 'Data_Acquisition_Method_Most_Popular_Genes', 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019', 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019', 'Descartes_Cell_Types_and_Tissue_2021', 'DisGeNET', 'Disease_Perturbations_from_GEO_down', 'Disease_Perturbations_from_GEO_up', 'Disease_Signatures

In [9]:
# set target libs to be used in GSEApy
target_libs = [
    # 'WikiPathway_2021_Human',
    # 'KEGG_2021_Human',
    # 'MSigDB_Hallmark_2020',
    # 'GO_Biological_Process_2021',
    # 'GO_Cellular_Component_2021',
    # 'GO_Molecular_Function_2021',
    # 'HDSigDB_Human_2021',
    # 'Jensen_COMPARTMENTS'
    'TF-LOF_Expression_from_GEO'
    ]


# careful here, if i use test as description, it works; if i use 'nc_B14_vs_TGF and kd_B14_vs_TGF' as description, it will throw error on gp.enrichr method
# description = 'nc_B14_vs_TGF and kd_B14_vs_TGF'
description = 'test'

In [10]:
for gene_list_name, gene_list in dict_gene_list.items():
    print(gene_list_name)
    # print(gene_list)
    try :
        enr = gp.enrichr(
                    gene_list=gene_list,
                    gene_sets=target_libs,
                    organism='Human',
                    description=description,
                    outdir=wk_dir+'enrichr_plots/'+gene_list_name,
                    # no_plot=False,
                    cutoff=0.05,
                    verbose=True
                    )
        # print(enr.res2d)   
        # time.sleep(10)
    except e as Exception:
        print(f'error: {e}')

    # to save your figure, make sure that ``ofname`` is not None
    title = ' and '.join(target_libs) + ' ' + gene_list_name
    print(title)

    try :
        
        dotplot(enr.res2d,title=title,
            top_term=20,
            cmap='viridis_r',
            ofname=wk_dir + title+'.png',
            )
        # time.sleep(10)                
    except e2 as Exception:
        print(f'error2: {e2}')

B14_KD_vs_NC_UP_genes


2022-04-21 19:24:51,455 Connecting to Enrichr Server to get latest library names
2022-04-21 19:24:51,456 Analysis name: test, Enrichr Library: TF-LOF_Expression_from_GEO
2022-04-21 19:24:59,149 Save file of enrichment results: Job Id:da5b737cb78f16fa9d4b1ee13775bee5
2022-04-21 19:24:59,292 Done.



TF-LOF_Expression_from_GEO B14_KD_vs_NC_UP_genes
Legend is shown.localtion  best


In [ ]:
# same as above, for single data plot
for gene_list_name, gene_list in dict_gene_list.items():
    print(gene_list_name)

name = 'kd_B14_vs_TGFb_UP_genes'
list = dict_gene_list[name]
enr = gp.enrichr(
                gene_list=list,
                gene_sets=target_libs,
                organism='Human',
                description=description,
                outdir=wk_dir+'enrichr_plots/'+name,
                # no_plot=False,
                cutoff=0.05,
                verbose=True
                )
# print(enr.res2d)   


# to save your figure, make sure that ``ofname`` is not None
title = ' and '.join(target_libs) + ' ' + name
# print(title)

dotplot(enr.res2d,title=title,
        top_term=20,
        cmap='viridis_r',
        # ofname=wk_dir + title+'.png',
        )   